In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import json
import jsonlines

/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NER = ["EFFECT"]
set = "test"

In [3]:
l = []
for ner in NER:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [4]:
df = df.reset_index().drop(columns=["index"])

In [5]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[In, a, second, experiment, ,, the, G., rostoc...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.6364471912384033, -2.252444505691528, 5.0...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, 2, -100, -10...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 7, 7, 7, 7, 8, ...",EFFECT
1,"[The, major, menaquinones, of, AN20, ^T, were,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.9053657054901123, -2.812688827514648, 5.5...","[-100, 2, 2, 2, -100, -100, -100, 2, 2, -100, ...","[None, 0, 1, 2, 2, 2, 2, 3, 4, 4, 5, 5, 6, 7, ...",EFFECT
2,"[Plasmids, to, complement, Δ, flgX, strains, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-2.666915655136108, -2.55630612373352, 5.215...","[-100, 2, 2, 2, 2, 2, -100, -100, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 9, ...",EFFECT
3,"[At, 120, h, ,, the, sporulation, rates, range...","[O, O, O, O, O, B, I, O, O, O, O, O, O, O, O, ...",3,"[[-2.706206560134887, -2.248944759368896, 4.97...","[-100, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, -100, -10...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 10,...",EFFECT
4,"[Wolbachia, infection, in, D., melanogaster, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-2.508585691452026, -2.584029912948608, 5.10...","[-100, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11...",EFFECT
...,...,...,...,...,...,...,...
619,"[Five, representative, A., veronii, isolates, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",619,"[[-2.685906171798706, -3.16297435760498, 6.072...","[-100, 2, 2, 2, -100, 2, -100, -100, 2, 2, 2, ...","[None, 0, 1, 2, 2, 3, 3, 3, 4, 5, 6, 7, 7, 8, ...",EFFECT
620,"[The, type, strain, of, P., pacaensis, sp., no...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",620,"[[-2.348237991333008, -2.974963903427124, 5.41...","[-100, 2, 2, 2, 2, 2, -100, 2, -100, -100, -10...","[None, 0, 1, 2, 3, 4, 4, 5, 5, 5, 5, 6, 6, 7, ...",EFFECT
621,"[Of, the, 16, proteins, classified, as, essent...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",621,"[[-2.480073451995849, -2.5230202674865723, 5.2...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
622,"[The, strain, 16-SW-7, was, isolated, from, a,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",622,"[[-2.666139125823974, -2.914419412612915, 5.92...","[-100, 2, 2, 2, -100, -100, -100, -100, 2, 2, ...","[None, 0, 1, 2, 2, 2, 2, 2, 3, 4, 5, 6, 7, 8, ...",EFFECT


In [6]:
label_ids = df[["label_ids"]]

In [7]:
annotations = []
for j in range(len(df)):
	for i in zip(df.loc[j,"tokens"],df.loc[j,"ner_tags"]):
		if i[1] != "O":
			annotations.append({"label":i[1],"text":i[0]})

### Predict sentences again

In [8]:
ner = "EFFECT"

In [9]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=0)


In [10]:
# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

predictions = []
# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:
            # Get the value from the "text" field
            text = l["data"]["text"]
            # Pass the value to the nlp() function
            print(text)
            predict = nlp(text)
            predictions.append(predict)
            print(predict)

In a second experiment, the G. rostochiensis population ‘HAR1’ was used to test the ability of the newly formed cysts to hatch without diapause.
[]
The major menaquinones of AN20 ^T were MK-5 (60.8%) and MK-11 (29.32%), while the major menaquinones of AN421 ^T and AN502 were MK-5 (58.5% and 55.1%, respectively) and MK-10 (35.5% and 40.1%, respectively).
[]
Plasmids to complement Δ flgX strains were constructed by amplifying flgX from codon 2 to the stop codon (with and without a FLAG tag attached to the 3′ end prior to the stop codon) from the 81-176 genome, followed by assembly into BamHI-digested pECO102 or pDAR964 to yield pDAR5009, pDAR5011, and pDAR5010.
[]
At 120 h, the sporulation rates ranged from 6.53 to 7.66 log10 CFU/mL, with RT883 and 027 producing similar amounts of spores.
[]
Wolbachia infection in D. melanogaster was first described as a single clonal infection, and for this reason it is still often referred to as a single Wolbachia strain-wMel.
[]
Among cellulolytic enz

/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[]
Mono-association of GF zebrafish with multiple commensal bacterial strains, including the gram-negative Aeromonas veronii strain ZOR0001 and Enterobacter cloacae strain ZOR0014, and gram-positive Staphylococcus sp.
[]
Bacillus safensis FO-36b and Bacillus pumilus SAFR-032: a whole genome comparison of two spacecraft assembly facility isolates
[]
Including: (i) Chryseobacterium camelliae Dolsongi-HT1, isolated from green tea leaves [ 48 ]; (ii) Chryseobacterium gallinarum strain DSM 27622, isolated from chicken [ 49 ]; and (iii) Chryseobacterium sp.
[]
CT3 was isolated from an activated sludge treatment plant in 1994 in Bari West, Italy (41°06′41″ N 16°51′19″ E) [ 30 ].
[]
Consistent with the estA and eltB mRNA fold changes, pSTa secretion and periplasmic, as well as secreted LT levels in the IMM07 and IMM96 strains were not significantly changed after incubation in heat-treated IEC-conditioned medium (Figure 5).
[]
Database searches for Drosphila simulans, Drosophila sechellia and D

In [11]:
predictions = [p for p in predictions if p]

In [12]:
predictions

[[{'entity_group': 'I',
   'score': 0.5108396,
   'word': 'import of inorganic phosphate',
   'start': 39,
   'end': 68}],
 [{'entity_group': 'I',
   'score': 0.4298396,
   'word': 'toxicity',
   'start': 123,
   'end': 131}],
 [{'entity_group': 'I',
   'score': 0.49852344,
   'word': 'bowel movement frequency',
   'start': 135,
   'end': 159}],
 [{'entity_group': 'I',
   'score': 0.38617444,
   'word': 'cell viability',
   'start': 54,
   'end': 68}],
 [{'entity_group': 'I',
   'score': 0.61834365,
   'word': 'infection on',
   'start': 49,
   'end': 61},
  {'entity_group': 'I',
   'score': 0.62816334,
   'word': 'lesion sizes',
   'start': 123,
   'end': 135}],
 [{'entity_group': 'B',
   'score': 0.3494878,
   'word': 'somatic',
   'start': 134,
   'end': 141},
  {'entity_group': 'I',
   'score': 0.39658174,
   'word': 'embryos',
   'start': 142,
   'end': 149}],
 [{'entity_group': 'I',
   'score': 0.51881105,
   'word': 'limitation',
   'start': 34,
   'end': 44}],
 [{'entity_group'

In [13]:
annotations

[{'label': 'B', 'text': 'sporulation'},
 {'label': 'I', 'text': 'rates'},
 {'label': 'B', 'text': 'import'},
 {'label': 'I', 'text': 'of'},
 {'label': 'I', 'text': 'inorganic'},
 {'label': 'I', 'text': 'phosphate'},
 {'label': 'I', 'text': 'across'},
 {'label': 'I', 'text': 'the'},
 {'label': 'I', 'text': 'membrane'},
 {'label': 'B', 'text': 'adjusted'},
 {'label': 'I', 'text': 'to'},
 {'label': 'I', 'text': '1'},
 {'label': 'I', 'text': '×'},
 {'label': 'I', 'text': '10'},
 {'label': 'I', 'text': '^8'},
 {'label': 'I', 'text': 'CFU'},
 {'label': 'I', 'text': '/'},
 {'label': 'I', 'text': 'ml'},
 {'label': 'B', 'text': 'reactivated'},
 {'label': 'I', 'text': 'at'},
 {'label': 'I', 'text': '28°C'},
 {'label': 'B', 'text': 'high'},
 {'label': 'I', 'text': 'toxicity'},
 {'label': 'B', 'text': 'obligate'},
 {'label': 'I', 'text': 'symbioses'},
 {'label': 'B', 'text': 'stool'},
 {'label': 'I', 'text': 'condition'},
 {'label': 'B', 'text': 'bowel'},
 {'label': 'I', 'text': 'movement'},
 {'la

In [14]:
for prediction in predictions:
    pred = prediction[0]["word"]
    count = 0
    for annotation in annotations:
        ann = annotation["text"]
        if ann in pred:
            count +=1
    if count <1:
        print(pred)

siderophores
selective pressures
cyst
susceptibility
teer
v


In [15]:
ners =["COMPOUND", "DISEASE", "EFFECT","ISOLATE","MEDIUM","METABOLITE","ORGANISM",
            "PHENOTYPE","PROTEIN","SPECIES","STRAIN"]
for ner in ners:
    test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
    test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
    merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
    merged.loc[:,"ner"] = ner
    label_ids = merged[["label_ids"]]

    annotations = []
    for j in range(len(merged)):
    	for i in zip(merged.loc[j,"tokens"],merged.loc[j,"ner_tags"]):
    		if i[1] != "O":
    			annotations.append({"label":i[1],"text":i[0]})
    
    path= f"../NER_output/{ner}"
    
    tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
    model = AutoModelForTokenClassification.from_pretrained(path)
    
    nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
                   aggregation_strategy="average", device=0)

    # Specify the path to the JSONL file
    jsonl_file = f"../NER/{ner}/test.jsonls"
    
    predictions = []
    # Open the JSONL file
    with jsonlines.open(jsonl_file) as reader:
        # Iterate over each line in the file
        for line in reader:
            for l in line:
    
                # Get the value from the "text" field
                text = l["data"]["text"]
                # Pass the value to the nlp() function
                predict = nlp(text)
                predictions.append(predict)
    predictions = [p for p in predictions if p]
    print("-----")
    print(ner)
    print("-----")
    for prediction in predictions:
        pred = prediction[0]["word"]
        count = 0
        for annotation in annotations:
            ann = annotation["text"].lower()
            if ann in pred:
                count +=1
        if count ==0:
            print(pred)

/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
COMPOUND
-----
na
n
ferric
_
sulfide
cc
5


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
DISEASE
-----
acute
7th
biltong
tuberculosis
restrictive anorexia nervosa
scrapie
fibrosis
non
abortion
leptospirosis
gastric
postinjury urinary
oral
melanoma
pathological thrombosis
blast
fop


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

-----
PROTEIN
-----
ll
opn4
p
enoyl
β
mn
mys
endo
s
o
p
n
m
β


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
SPECIES
-----
l
p
i
v
k
r
r
p
d
c
g
m
p
p
m
m
c
p
l
l
d
p
p
l
n
a
a
l
n
k
p
a
m
l
p
l
r
p
l
l
f
r
p


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
STRAIN
-----
x
l
lz
p
l
v
p
v
y
l
p
h
s
s
v
n
v
s
v
l
81
l
x
p
s
p
v
s
l
s
p
p
pid21
s
s
n
t
v
tg78
v
p
"
l
h
p
dq12
p
g
p
17
p
y
v
osish
p
s
p
v
l
l
dq12
p
pti
y
v
dq12
